# Unet Rust detection

Authors: Kevin, Gabriel, Charles

Description: This code is created for pre processing the masks and the data related to Rust detection

In [1]:
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

# Importing section
from tensorflow.keras import models
from keras import layers
from tensorflow.keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
import numpy as np # IMPORTANT for the task 1
from keras import backend as K
import os
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
# Paper Correction
from keras import metrics
# End importing section
import cv2 # IMPORTANT for the task 1
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from scipy.io import wavfile
import pylab
from tensorflow.keras.utils import to_categorical
datagen = ImageDataGenerator(rescale=1)
batch_size = 32
from keras.layers import LeakyReLU, Conv2D, Input, BatchNormalization, Activation, Dense, Dropout, Conv2DTranspose, concatenate

In [2]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
import tensorflow as tf
from cityscapesscripts.helpers.labels import trainId2label as t2l

# Functions used in this project

In [10]:
def get_images_labels_features(directory, sample_count, new_image_shape = (512, 512)):
    '''Loads and crop the images according the appropriate conditions. Also, it returns the feature and labels of the VGG16 predictions'''
        
    images = np.zeros(shape=(sample_count, new_image_shape[0], new_image_shape[1], 3))    
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                            target_size = new_image_shape,
                                            batch_size = batch_size,
                                            class_mode = 'binary',
                                            shuffle = True,
                                            seed=0)
    i = 0

    for inputs_batch, labels_batch in generator:
                            
        images[i * batch_size : (i + 1) * batch_size] = inputs_batch        
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1

        if i * batch_size >= sample_count:

             break
    return(images, labels)

def get_train_test_data(base_dir = 'input_data/RustData',
                        train_sample_size = 45, test_sample_size = 45,  
                        new_image_shape = (512, 512), 
                        rescale = 1):
    '''This function imports the dataset and split into train and test data return the features and labels of the images'''
        
    
    # Importing images data
    train_dir = os.path.join(base_dir, 'train')
    test_dir = os.path.join(base_dir, 'test')
    datagen = ImageDataGenerator(rescale=rescale)
    batch_size = 20    

    # Getting data features and labels    
    train_information = get_images_labels_features(directory = train_dir, 
                                                   sample_count = train_sample_size, 
                                                   new_image_shape = new_image_shape)    
    train_images = train_information[0]
    train_labels = train_information[1]
    
    test_information = get_images_labels_features(directory = test_dir, 
                                                  sample_count = test_sample_size, 
                                                  new_image_shape = new_image_shape)    
    test_images = test_information[0]
    test_labels = test_information[1]

    # Preparing data features    
    train_labels = to_categorical(train_labels)
    test_labels = to_categorical(test_labels)        
        
    return(train_images, train_labels, test_images, test_labels)

In [3]:
def get_images_labels_features(directory, sample_count, new_image_shape = (512, 512)):
    '''Loads and crop the images according the appropriate conditions. Also, it returns the feature and labels of the VGG16 predictions'''
        
    images = np.zeros(shape=(sample_count, new_image_shape[0], new_image_shape[1], 3))    
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                            target_size = new_image_shape,
                                            batch_size = batch_size,
                                            class_mode = 'binary',
                                            shuffle = True,
                                            seed=0)
    i = 0

    for inputs_batch, labels_batch in generator:
                            
        images[i * batch_size : (i + 1) * batch_size] = inputs_batch        
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1

        if i * batch_size >= sample_count:

             break
    return(images, labels)


In [11]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(6, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

# Load the images

In [ ]:
train_images, train_labels, test_images, test_labels = get_train_test_data(base_dir = 'input_data/RustData/',
                                                                           train_sample_size = 45, test_sample_size = 45, 
                                                                           new_image_shape = (512, 512))


In [ ]:
train_images_maks, train_labels_maks, test_images_maks, test_labels_maks = get_train_test_data(base_dir = 'input_data/RustDataMasks/',
                                                                               train_sample_size = 45, test_sample_size = 45, 
                                                                               new_image_shape = (512, 512))


# Create the model

In [12]:
input_img = Input((512, 512, 3), name='img')
my_model = get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True)
my_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 512, 512, 16  448         ['img[0][0]']                    
                                )                                                                 
                                                                                                  
 batch_normalization_19 (BatchN  (None, 512, 512, 16  64         ['conv2d_20[0][0]']              
 ormalization)                  )                                                           

 spose)                         )                                                                 
                                                                                                  
 concatenate_5 (Concatenate)    (None, 128, 128, 12  0           ['conv2d_transpose_5[0][0]',     
                                8)                                'activation_23[0][0]']          
                                                                                                  
 dropout_13 (Dropout)           (None, 128, 128, 12  0           ['concatenate_5[0][0]']          
                                8)                                                                
                                                                                                  
 conv2d_32 (Conv2D)             (None, 128, 128, 64  73792       ['dropout_13[0][0]']             
                                )                                                                 
          

# Train the model

In [13]:
callbacks = [    
    ModelCheckpoint('model-unet-rust.h5',verbose = 1, save_best_only = True, 
                    save_weights_only = True, monitor='val_acc')
]

In [ ]:
my_model.compile(optimizer=Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                 metrics=["acc"])
results = my_model.fit(train_images, train_images_maks[:, :, :, 0:1], 
                    batch_size=32, epochs=30, callbacks=callbacks,
                    validation_data=(test_images[:, :, :, :], test_images_maks[:, :, :, 0:1]))

In [ ]:
exploratory_performance = pd.DataFrame(results.history)
#exploratory_performance.to_csv("../../output_data/unet_dataset.csv")

In [ ]:
exploratory_performance.plot()